## KÖNYVTÁR IMPORTOK

In [223]:
# !pip install sentence-transformers
# !pip install hdbscan

## ADAT IMPORT

In [224]:
import pandas as pd
data = pd.read_csv('/Users/tamasmakos/dev/k-monitor/Export_KMONITOR.csv')

In [225]:
# Dropping rows where Ajánlatkérő szervezet neve OR Nyertes ajánlattevő neve is empty
data = data.dropna(subset=['Ajánlatkérő szervezet neve', 'Nyertes ajánlattevő neve'])

### VÁLTOZÓK LEVÁLASZTÁSA

In [226]:
must_and_nice_to_have_columns = [
    'Eljárás EKR azonosító',
    'Ajánlatkérő szervezet neve',
    'Ajánlatkérő nemzeti azonosítószáma',
    'Ajánlatkérő szervezet típusa',
    'Ajánlatkérő szervezet főtevékenysége',
    'Eljárásrend',
    'Eljárás fajtája',
    'Eljárás tárgya',
    'Szerződés típusa',
    'A beszerzés végleges összértéke',
    'A beszerzés végleges összértéke pénznem',
    'Teljesítés helye NUTS-kód(ok)',
    'Szerződés megkötésének dátuma',
    'Beérkezett ajánlatok száma',
    'Nyertes ajánlattevő adószáma (adóazonosító jele)',
    'Nyertes ajánlattevő neve',
    'Eljárás tárgya',
    'Fő CPV-kód(ok)',
    'További CPV-kód(ok)',
    'A beszerzés európai uniós alapokból finanszírozott projekttel és/vagy programmal kapcsolatos',
    'Szerződés/rész odaítélésre került',
    'Nyertes ajánlattevő kkv',
    'Szerződés megkötésének dátuma',
    'Hirdetmény közzétételének dátuma',
    'Minőségi kritérium alkalmazásra került',
    'Költség kritérium alkalmazásra került',
    'Ár kritérium alkalmazásra került',
    'A beszerzés európai uniós alapokból finanszírozott projekttel és/vagy programmal kapcsolatos',
    'Szerződés/rész odaítélésre került',
    'Alvállalkozók igénybevétele a szerződés teljesítéséhez',
    'Alvállalkozó(k) megnevezése, adószáma'
]


In [227]:
data = data[must_and_nice_to_have_columns]

In [228]:
data.shape

(72569, 31)

# SZERZŐDÉSEK KIBONTÁSA

- Nyertes ajánlattevő kkv
- Nyertes ajánlattevő adószáma (adóazonosító jele)

In [229]:
def process_contracts(df, issuer_name_col, issuer_id_col, winner_name_col, winner_id_col, kkv_col):
    # Reset the index of the DataFrame to ensure uniqueness
    df = df.reset_index(drop=True)

    def safe_strip(value):
        """ Safely strip strings, handling NaN values """
        return value.strip() if isinstance(value, str) else value

    # Creating a new DataFrame to hold the expanded data
    expanded_data = []

    for _, row in df.iterrows():
        issuer_names = row[issuer_name_col].split('|') if pd.notna(row[issuer_name_col]) else ['']
        issuer_ids = row[issuer_id_col].split('|') if pd.notna(row[issuer_id_col]) else ['']
        winner_names = row[winner_name_col].split('|') if pd.notna(row[winner_name_col]) else ['']
        winner_ids = row[winner_id_col].split('|') if pd.notna(row[winner_id_col]) else ['']
        kkv_values = row[kkv_col].split('|') if pd.notna(row[kkv_col]) else ['']

        for issuer_name, issuer_id in zip(issuer_names, issuer_ids):
            for winner_name, winner_id, kkv in zip(winner_names, winner_ids, kkv_values):
                if safe_strip(issuer_name) and safe_strip(winner_name):
                    # Convert row to dictionary to avoid copying the index
                    new_row = row.to_dict()
                    new_row[issuer_name_col] = safe_strip(issuer_name)
                    new_row[issuer_id_col] = safe_strip(issuer_id)
                    new_row[winner_name_col] = safe_strip(winner_name)
                    new_row[winner_id_col] = safe_strip(winner_id)
                    new_row[kkv_col] = safe_strip(kkv)
                    expanded_data.append(new_row)

    return pd.DataFrame(expanded_data)

# Example usage
data = process_contracts(
    data,
    'Ajánlatkérő szervezet neve',
    'Ajánlatkérő nemzeti azonosítószáma',
    'Nyertes ajánlattevő neve',
    'Nyertes ajánlattevő adószáma (adóazonosító jele)',
    'Nyertes ajánlattevő kkv',
)


In [230]:
data.head(10)

,Eljárás EKR azonosító,Ajánlatkérő szervezet neve,Ajánlatkérő nemzeti azonosítószáma,Ajánlatkérő szervezet típusa,Ajánlatkérő szervezet főtevékenysége,Eljárásrend,Eljárás fajtája,Eljárás tárgya,Szerződés típusa,A beszerzés végleges összértéke,A beszerzés végleges összértéke pénznem,Teljesítés helye NUTS-kód(ok),Szerződés megkötésének dátuma,Beérkezett ajánlatok száma,Nyertes ajánlattevő adószáma (adóazonosító jele),Nyertes ajánlattevő neve,Fő CPV-kód(ok),További CPV-kód(ok),A beszerzés európai uniós alapokból finanszírozott projekttel és/vagy programmal kapcsolatos,Szerződés/rész odaítélésre került,Nyertes ajánlattevő kkv,Hirdetmény közzétételének dátuma,Minőségi kritérium alkalmazásra került,Költség kritérium alkalmazásra került,Ár kritérium alkalmazásra került,Alvállalkozók igénybevétele a szerződés teljesítéséhez,"Alvállalkozó(k) megnevezése, adószáma"
0,EKR001530432019,Albert Schweitzer Kórház- Rendelőintézet,15813774210,U - Egyéb típus:,KL - Egészségügy,Uniós eljárásrend,Nyílt eljárás,Traumatológiai implantátumok beszerzése,Árubeszerzés,110810613.0,HUF,HU312,2020-07-14,1.0,"11172790210,HU11172790",Sanatmetal Ortopédiai és Traumatológiai Eszköz...,33184100-4,33184100-4,Nem,Igen,Igen,2020-07-23,Nem,Nem,Igen,Nem,NaN
1,EKR001531082019,Markhot Ferenc oktatókórház és Rendelőintézet,15813839210,U - Egyéb típus:,KL - Egészségügy,Uniós eljárásrend,Nyílt eljárás,Röntgen kontrasztanyagok beszerzése,Árubeszerzés,52368257.0,HUF,HU312,2020-09-17,1.0,"25560611241,HU25560611",Clear Pharma Zrt.,33696800-3,33696800-3,Nem,Igen,Igen,2020-09-27,Nem,Nem,Igen,Nem,AIRMAX Cargo Budapest Zrt. (adószáma: 24085984...
2,EKR001531392019,"BMSK Beruházási, Műszaki Fejlesztési, Sportüze...",12464780251,N - Közjogi szervezet,KL - Egyéb tevékenység:,Nemzeti eljárásrend,Kbt. 112. § (1) bekezdés b) pont Nyílt eljárás...,Tanuszoda kivitelezése Mosonmagyaróváron,Építési beruházás,788155483.0,HUF,HU221,2020-12-02,4.0,26739919243,Falcon Global Zártkörűen Működő Részvénytársaság,45000000-7,45232100-3|45231400-9|45231300-8|45231220-3|71...,Nem,Igen,Igen,2020-12-08,Igen,Nem,Igen,Igen,3C Computers and Communivations Consultans (ad...
3,EKR001531392019,"BMSK Beruházási, Műszaki Fejlesztési, Sportüze...",12464780251,N - Közjogi szervezet,KL - Egyéb tevékenység:,Nemzeti eljárásrend,Kbt. 112. § (1) bekezdés b) pont Nyílt eljárás...,Tanuszoda kivitelezése Mosonmagyaróváron,Építési beruházás,788155483.0,HUF,HU221,2020-12-02,4.0,"12525689241,HU12525689",OCTOPUS Invest Szolgáltató Korlátolt Felelőssé...,45000000-7,45232100-3|45231400-9|45231300-8|45231220-3|71...,Nem,Igen,Igen,2020-12-08,Igen,Nem,Igen,Igen,3C Computers and Communivations Consultans (ad...
4,EKR001531532019,IKK Innovatív Képzéstámogató Központ Zrt.,24195162241,U - Egyéb típus:,KL - Egyéb tevékenység:,Uniós eljárásrend,Nyílt eljárás,Módszertani javaslatok kidolgozása,Szolgáltatás megrendelés,786880000.0,HUF,HU110,2020-03-10,4.0,"11782454241,HU 11782454",Stratis Vezetői és Informatikai Tanácsadó Kft.,71621000-7,71621000-7,Igen,Igen,Igen,2020-03-31,Igen,Nem,Igen,Igen,"Csernoch Mária, 4029 Debrecen Kígyó u. 36/b 83..."
5,EKR001531082019,Markhot Ferenc oktatókórház és Rendelőintézet,15813839210,U - Egyéb típus:,KL - Egészségügy,Uniós eljárásrend,Nyílt eljárás,Röntgen kontrasztanyagok beszerzése,Árubeszerzés,52368257.0,HUF,HU312,2020-09-11,1.0,"25567072241,HU25567072",Atlas Pharma Gyógyszernagykereskedelmi Korláto...,33696800-3,33696800-3,Nem,Igen,Nem,2020-09-27,Nem,Nem,Igen,Nem,Euromedic International Hungária Kft. (adószám...
6,EKR001531082019,Markhot Ferenc oktatókórház és Rendelőintézet,15813839210,U - Egyéb típus:,KL - Egészségügy,Uniós eljárásrend,Nyílt eljárás,Röntgen kontrasztanyagok beszerzése,Árubeszerzés,52368257.0,HUF,HU312,2020-09-11,2.0,"25567072241,HU25567072",Atlas Pharma Gyógyszernagykereskedelmi Korláto...,33696800-3,33696800-3,Nem,Igen,Nem,2020-09-27,Nem,Nem,Igen,Nem,Euromedic International Hungária Kft. (adószám...
7,EKR001530502019,Semmelweis Egyetem,15329808242,U - Egyéb típus:,KL - Egyéb tev

In [231]:
data.shape

(101135, 27)

# GLOBÁLIS FUNKCIÓK

## ÁLTALÁNOS TISZTÍTÓ

In [232]:
import re

def general_text_cleaning(df, column_name):
    """
    Cleans the text in the specified column of a DataFrame.

    Parameters:
    df (DataFrame): The dataframe to be processed.
    column_name (str): The name of the column to clean.

    Returns:
    DataFrame: The dataframe with cleaned text in the specified column.
    """

    # Making a copy of the dataframe to avoid modifying the original data
    cleaned_df = df.copy()

    # Function to clean individual text entries
    def clean_text(text):
        # Ensuring that the input is a string
        if not isinstance(text, str):
            if pd.isna(text):
                return "Ismeretlen"  # Replace NaN with "Ismeretlen"
            else:
                return str(text)  # Convert non-string, non-NaN values to string

        # Proceed with cleaning
        text = text.strip()  # Trimming white spaces
        text = re.sub(r'\s+', ' ', text)  # Normalizing whitespaces
        text = text.title()  # Standardizing capitalization
        text = re.sub(r'[^A-Za-z0-9 ÁÉÍÓÖŐÚÜŰáéíóöőúüű,.]', '', text)  # Removing special characters

        return text

    # Applying the cleaning function to the specified column
    cleaned_df[column_name] = cleaned_df[column_name].apply(clean_text)

    return cleaned_df

## VEKTOR ALAPÚ SZÖVEG CSOPORTOSÍTÁS

### VEKTORIZÁLÁS ÉS HASONLÓSÁG KERESÉS

In [233]:
import pandas as pd
import re
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Assuming general_text_cleaning function exists and is correctly implemented

def standardize_entries_with_log_and_similarity_enhanced(dataframe, name_column, type_column, threshold=0.799):
    # Ensure that both columns are treated as Series and handle NaN values
    dataframe[name_column] = dataframe[name_column].fillna("Ismeretlen")
    dataframe[type_column] = dataframe[type_column].fillna("")

    unique_values = dataframe[[name_column, type_column, 'unique_id']].drop_duplicates(subset=[name_column, type_column])

    # Combine the name and type columns for similarity check
    combined_column = unique_values[name_column] + " " + unique_values[type_column]
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(combined_column)
    cosine_similarities = linear_kernel(tfidf_matrix)

    similarity_groups = defaultdict(list)
    similarity_scores = {}

    for i in range(len(cosine_similarities)):
        for j in range(i + 1, len(cosine_similarities)):
            if cosine_similarities[i, j] >= threshold:
                entry_i = combined_column.iloc[i]
                entry_j = combined_column.iloc[j]
                similarity_groups[entry_i].append(entry_j)
                similarity_groups[entry_j].append(entry_i)
                similarity_scores[(entry_i, entry_j)] = cosine_similarities[i, j]
                similarity_scores[(entry_j, entry_i)] = cosine_similarities[i, j]

    standardized_entries = {}
    for entry, similar_entries in similarity_groups.items():
        if entry not in standardized_entries:
            standardized_entries[entry] = entry
            for similar_entry in similar_entries:
                standardized_entries[similar_entry] = entry

    change_log_data = []
    for i, row in unique_values.iterrows():
        combined_value = row[name_column] + " " + row[type_column]
        new_value = standardized_entries.get(combined_value, combined_value)
        if combined_value != new_value:
            similarity_score = similarity_scores.get((combined_value, new_value), 0)
            change_log_data.append({
                'Unique ID': row['unique_id'],
                'Original Value': combined_value,
                'New Value': new_value,
                'Similarity Score': similarity_score
            })

    # Map the standardized names back to the DataFrame
    standardized_map = {row[name_column] + " " + row[type_column]: standardized_entries.get(row[name_column] + " " + row[type_column], row[name_column] + " " + row[type_column]) for i, row in unique_values.iterrows()}
    dataframe['Standardized Name'] = dataframe.apply(lambda row: standardized_map[row[name_column] + " " + row[type_column]], axis=1)

    change_log_df = pd.DataFrame(change_log_data)
    return dataframe, change_log_df


# SZÖVEGES VÁLTOZÓK TISZTÍTÁSA

## AJÁNLATTEVŐ ÉS NYERTES NEVÉNEK KONSZOLIDÁLÁSA

In [234]:
import re
import pandas as pd

def standardize_company_type(company_name):
    """
    Standardize Hungarian company type abbreviation and extract it into a separate column.

    Args:
    company_name (str): The company name string to standardize.

    Returns:
    tuple: A tuple containing the cleaned company name and the company type abbreviation.
    """
    # Remove specific phrases before processing company types
    phrases_to_remove = ["Szolgáltató És Kereskedelmi", "Kereskedelmi És Szolgáltató", "Kereskedelmi", "Szolgáltató"]
    for phrase in phrases_to_remove:
        company_name = company_name.replace(phrase, "")

    company_name_lower = company_name.lower()

    # Regular expressions for company types
    company_types = {
        "Nonprofit Kft.": r"\bnonprofit\s*k\.?f\.?t\.?\b",
        "Kft.": r"\b(k\.?f\.?t\.?|korlátolt felelősségű társaság)\b",
        
        "Nonprofit Nyrt.": r"\bnonprofit\s*nyrt\.?\b",
        "Nyrt.": r"\bnyilvánosan\s*működő\s*(részvény)?\s*társaság\b|\bnyrt\.?\b",

        "Nonprofit Zrt.": r"\bnonprofit\s*zrt\.?\b",
        "Zrt.": r"\bzártkörűen\s*működő\s*(részvény)?\s*társaság\b|\bzrt\.?\b",

        "Nonprofit Rt.": r"\bnonprofit\s*rt\.?\b",
        "Rt.": r"\brészvénytársaság\b|\brt\.?\b",

        "Nonprofit EV": r"\bnonprofit\s*e\.?\s*v\.?\b",
        "EV": r"\b(e\.?\s*v\.?|egyéni vállalkozó)\b",

        "Nonprofit Bt.": r"\bnonprofit\s*b\.?t\.?\b",
        "Bt.": r"\b(b\.?t\.?|betéti társaság)\b",

        "Nonprofit Sz.": r"\bnonprofit\s*s\.?z\.?\b",
        "Sz.": r"\b(s\.?z\.?|szövetkezet)\b",

        "Nonprofit Kkt.": r"\bnonprofit\s*k\.?k\.?t\.?\b",
        "Kkt.": r"\b(k\.?k\.?t\.?|közkereseti társaság)\b"
    }

    # Identify company type
    for type_abbr, type_regex in company_types.items():
        if re.search(type_regex, company_name_lower):
            # Remove identified type from the name
            cleaned_name = re.sub(type_regex, "", company_name, flags=re.IGNORECASE)

            # Comprehensive cleaning to remove trailing whitespaces, periods, and commas
            cleaned_name = re.sub(r'[\s.,]+\Z', '', cleaned_name)

            return cleaned_name, type_abbr

    # Comprehensive cleaning for names where no type is identified
    cleaned_name = re.sub(r'[\s.,]+\Z', '', company_name)
    return cleaned_name, None

# Apply your general text cleaning function first
data = general_text_cleaning(data, 'Ajánlatkérő szervezet neve')
data = general_text_cleaning(data, 'Nyertes ajánlattevő neve')

# Then, apply the company type standardization function
data['Ajánlatkérő szervezet neve - tisztított'], data['Ajánlatkérő Cégtípus'] = zip(*data['Ajánlatkérő szervezet neve'].apply(standardize_company_type))
data['Nyertes ajánlattevő neve - tisztított'], data['Nyertes Cégtípus'] = zip(*data['Nyertes ajánlattevő neve'].apply(standardize_company_type))


In [235]:
text_cleaning = {'Ajánlatkérő szervezet neve - tisztított': 'Ajánlatkérő Cégtípus', 'Nyertes ajánlattevő neve - tisztított': 'Nyertes Cégtípus'}

In [236]:
import pandas as pd

# Assuming general_text_cleaning function exists and is correctly implemented

data['unique_id'] = data.index

for name_column, type_column in text_cleaning.items():
    # Handle NaN values in both columns
    data[name_column] = data[name_column].fillna("Ismeretlen")
    data[type_column] = data[type_column].fillna("")

    # Step 1: Clean the name column using general_text_cleaning
    cleaned_text_data = general_text_cleaning(data, name_column)

    # Step 2: Standardize entries and generate change_log_df
    cleaned_text_data, change_log_df = standardize_entries_with_log_and_similarity_enhanced(cleaned_text_data, name_column, type_column)

    # Update the original data with cleaned text data
    data[name_column] = cleaned_text_data[name_column]
    data[name_column] = cleaned_text_data['Standardized Name']

    # Step 3: Save the change_log_df to a CSV file
    change_log_filename = f'{name_column}_change_log.csv'
    change_log_df.to_csv(change_log_filename, index=False)

    print(f"Processed and saved change log for column '{name_column}' in file '{change_log_filename}'.")

print("All specified columns have been processed.")


Processed and saved change log for column 'Ajánlatkérő szervezet neve - tisztított' in file 'Ajánlatkérő szervezet neve - tisztított_change_log.csv'.
Processed and saved change log for column 'Nyertes ajánlattevő neve - tisztított' in file 'Nyertes ajánlattevő neve - tisztított_change_log.csv'.
All specified columns have been processed.


### Szerződés megkötésének dátuma és Hirdetmény közzétételének dátuma


Ki kéne számolni a hirdetmény és szerződés közötti időt és berakni egy új oszlopba

In [237]:
hirdetmeny_datum = ['Hirdetmény közzétételének dátuma']
szerzodes_datum = ['Szerződés megkötésének dátuma']

In [238]:
# Creating a new column that calculates the difference between the two dates
data['Hirdetmény közzétételének dátuma'] = pd.to_datetime(data['Hirdetmény közzétételének dátuma'])
data['Szerződés megkötésének dátuma'] = pd.to_datetime(data['Szerződés megkötésének dátuma'])

data['Hirdetmény és szerződés megkötés között eltelt idő'] = data['Szerződés megkötésének dátuma'] - data['Hirdetmény közzétételének dátuma']

In [239]:
data[['Hirdetmény közzétételének dátuma', 'Szerződés megkötésének dátuma', 'Hirdetmény és szerződés megkötés között eltelt idő']].head(10)


,Hirdetmény közzétételének dátuma,Szerződés megkötésének dátuma,Hirdetmény és szerződés megkötés között eltelt idő
0,2020-07-23,2020-07-14,-9 days
1,2020-09-27,2020-09-17,-10 days
2,2020-12-08,2020-12-02,-6 days
3,2020-12-08,2020-12-02,-6 days
4,2020-03-31,2020-03-10,-21 days
5,2020-09-27,2020-09-11,-16 days
6,2020-09-27,2020-09-11,-16 days
7,2020-03-30,2020-03-17,-13 days
8,2020-09-27,2020-09-11,-16 days
9,2020-09-27,2020-09-11,-16 days


In [240]:
pd.set_option('display.max_columns', None)
data.head(5)

,Eljárás EKR azonosító,Ajánlatkérő szervezet neve,Ajánlatkérő nemzeti azonosítószáma,Ajánlatkérő szervezet típusa,Ajánlatkérő szervezet főtevékenysége,Eljárásrend,Eljárás fajtája,Eljárás tárgya,Szerződés típusa,A beszerzés végleges összértéke,A beszerzés végleges összértéke pénznem,Teljesítés helye NUTS-kód(ok),Szerződés megkötésének dátuma,Beérkezett ajánlatok száma,Nyertes ajánlattevő adószáma (adóazonosító jele),Nyertes ajánlattevő neve,Fő CPV-kód(ok),További CPV-kód(ok),A beszerzés európai uniós alapokból finanszírozott projekttel és/vagy programmal kapcsolatos,Szerződés/rész odaítélésre került,Nyertes ajánlattevő kkv,Hirdetmény közzétételének dátuma,Minőségi kritérium alkalmazásra került,Költség kritérium alkalmazásra került,Ár kritérium alkalmazásra került,Alvállalkozók igénybevétele a szerződés teljesítéséhez,"Alvállalkozó(k) megnevezése, adószáma",Ajánlatkérő szervezet neve - tisztított,Ajánlatkérő Cégtípus,Nyertes ajánlattevő neve - tisztított,Nyertes Cégtípus,unique_id,Hirdetmény és szerződés megkötés között eltelt idő
0,EKR001530432019,Albert Schweitzer Kórház Rendelőintézet,15813774210,U - Egyéb típus:,KL - Egészségügy,Uniós eljárásrend,Nyílt eljárás,Traumatológiai implantátumok beszerzése,Árubeszerzés,110810613.0,HUF,HU312,2020-07-14,1.0,"11172790210,HU11172790",Sanatmetal Ortopédiai És Traumatológiai Eszköz...,33184100-4,33184100-4,Nem,Igen,Igen,2020-07-23,Nem,Nem,Igen,Nem,NaN,Albert Schweitzer Kórház Rendelőintézet,,Sanatmetal Ortopédiai És Traumatológiai Eszköz...,Kft.,0,-9 days
1,EKR001531082019,Markhot Ferenc Oktatókórház És Rendelőintézet,15813839210,U - Egyéb típus:,KL - Egészségügy,Uniós eljárásrend,Nyílt eljárás,Röntgen kontrasztanyagok beszerzése,Árubeszerzés,52368257.0,HUF,HU312,2020-09-17,1.0,"25560611241,HU25560611",Clear Pharma Zrt.,33696800-3,33696800-3,Nem,Igen,Igen,2020-09-27,Nem,Nem,Igen,Nem,AIRMAX Cargo Budapest Zrt. (adószáma: 24085984...,Markhot Ferenc Oktatókórház És Rendelőintézet,,Clear Pharma Zrt.,Zrt.,1,-10 days
2,EKR001531392019,"Bmsk Beruházási, Műszaki Fejlesztési, Sportüze...",12464780251,N - Közjogi szervezet,KL - Egyéb tevékenység:,Nemzeti eljárásrend,Kbt. 112. § (1) bekezdés b) pont Nyílt eljárás...,Tanuszoda kivitelezése Mosonmagyaróváron,Építési beruházás,788155483.0,HUF,HU221,2020-12-02,4.0,26739919243,Falcon Global Zártkörűen Működő Részvénytársaság,45000000-7,45232100-3|45231400-9|45231300-8|45231220-3|71...,Nem,Igen,Igen,2020-12-08,Igen,Nem,Igen,Igen,3C Computers and Communivations Consultans (ad...,"Bmsk Beruházási, Műszaki Fejlesztési, Sportüze...",Zrt.,Falcon Global Zrt.,Zrt.,2,-6 days
3,EKR001531392019,"Bmsk Beruházási, Műszaki Fejlesztési, Sportüze...",12464780251,N - Közjogi szervezet,KL - Egyéb tevékenység:,Nemzeti eljárásrend,Kbt. 112. § (1) bekezdés b) pont Nyílt eljárás...,Tanuszoda kivitelezése Mosonmagyaróváron,Építési beruházás,788155483.0,HUF,HU221,2020-12-02,4.0,"12525689241,HU12525689",Octopus Invest Szolgáltató Korlátolt Felelőssé...,45000000-7,45232100-3|45231400-9|45231300-8|45231220-3|71...,Nem,Igen,Igen,2020-12-08,Igen,Nem,Igen,Igen,3C Computers and Communivations Consultans (ad...,"Bmsk Beruházási, Műszaki Fejlesztési, Sportüze...",Zrt.,Octopus Invest Kft.,Kft.,3,-6 days
4,EKR001531532019,Ikk Innovatív Képzéstámogató Központ Zrt.,24195162241,U - Egyéb típus:,KL - Egyéb tevékenység:,Uniós eljárásrend,Nyílt eljárás,Módszertani javaslatok kidolgozása,Szolgáltatás megrendelés,786880000.0,HUF,HU110,2020-03-10,4.0,"11782454241,HU 11782454",Stratis Vezetői És Informatikai Tanácsadó Kft.,71621000-7,71621000-7,Igen,Igen,Igen,2020-03-31,Igen,Nem,Igen,Igen,"Csernoch Mária, 4029 Debrecen Kígyó u. 36/b 83...",Ikk Innovatív Képzéstámogató Központ Zrt.,Zrt.,Stratis Vezetői És Informatikai Tanácsadó Kft.,Kft.,4,-21 days


In [241]:
# Dropping EUR currency rows in A beszerzés végleges összértéke pénznem
data = data[data['A beszerzés végleges összértéke pénznem'] != 'EUR']

In [242]:
# Dropping További CPV-kód(ok) column
data = data.drop(columns=['További CPV-kód(ok)'])

In [243]:
# Dropping Alvállalkozók igénybevétele a szerződés teljesítéséhez	Alvállalkozó(k) megnevezése, adószámak columns
data = data.drop(columns=['Alvállalkozók igénybevétele a szerződés teljesítéséhez', 'Alvállalkozó(k) megnevezése, adószáma'])

In [244]:
data['Szerződés típusa'].value_counts().sort_values(ascending=False)


Szerződés típusa
Árubeszerzés                44668
Szolgáltatás megrendelés    29616
Építési beruházás           24738
Name: count, dtype: int64

In [245]:
# Saving the cleaned data to a CSV file
data.to_csv('cleaned_data.csv', index=False)

### Változtatások visszaállítása

In [151]:
def revert_changes(dataframe, change_log_df, unique_ids):
    """
    Reverts changes for specific unique_ids based on the change log.

    Args:
    dataframe (pd.DataFrame): The main DataFrame where changes have been made.
    change_log_df (pd.DataFrame): The DataFrame containing the change log.
    unique_ids (list): List of unique_ids to revert changes for.

    Returns:
    pd.DataFrame: The DataFrame after reverting the specified changes.
    """
    # Filter the change log to get the rows for the given unique_ids
    revert_rows = change_log_df[change_log_df['Unique ID'].isin(unique_ids)]

    # Revert changes in the main DataFrame
    for _, row in revert_rows.iterrows():
        unique_id = row['Unique ID']
        original_value = row['Original Value']

        # Find the index in the main DataFrame to revert
        index_to_revert = dataframe[dataframe['unique_id'] == unique_id].index

        # Revert the 'Standardized Name' column to 'Original Value'
        dataframe.at[index_to_revert, 'Standardized Name'] = original_value

    return dataframe

# Example usage
# unique_ids_to_revert = [1, 2, 3]  # Replace with actual unique IDs
# data = revert_changes(data, change_log_df, unique_ids_to_revert)
